# Fabric Model Analysis & Optimization Toolkit

## Overview
This notebook provides comprehensive functions for analyzing semantic models to identify unused objects (measures, columns, and tables) that can be removed to optimize model performance and reduce complexity.

## Key Features
- **Unused Measures Detection**: Identifies measures not referenced in calculations or reports
- **Unused Columns Detection**: Finds columns not used in calculations or visualizations
- **Unused Tables Detection**: Locates tables with no references in the model
- **Relationship Analysis**: Examines table relationships for optimization opportunities

## Required Libraries
- `SemPy` which is part of the `semantic-link` feature with Core Fabric semantic model operations

---

## Installation & Dependencies

Install the required semantic-link-labs package for extended Fabric analytics capabilities.

In [ ]:
# Install semantic-link-labs for extended Fabric analytics
!pip install semantic-link-labs

## Library Imports & Configuration

Import essential libraries for Fabric model analysis:
- **`sempy.fabric`**: Primary interface for Fabric operations
- **`sempy_labs.report`**: Report analysis and object extraction
- **`pandas`**: Data manipulation framework
- **`matplotlib.pyplot`**: Data visualization and charting


In [ ]:
# Import core libraries for Fabric model analysis
import pandas as pd
import matplotlib.pyplot as plt
import sempy.fabric as fabric
from sempy_labs.report import ReportWrapper

# Configuration: Update these values for your specific workspace and dataset.
WORKSPACE = "Test Workspace"
DATASET = "New Waziri Dashboard Report"
REPORT = "New Waziri Dashboard Report"  # Optional: Specify if report name is different from dataset name

log_telemetry: sempy.relationships
log_telemetry: sempy.dependencies


C:\Users\DELL\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'azure'

# Data Preparation Functions

## Step 1: Get Model Dependencies & Report Objects

### Libraries Used:
- `fabric.get_model_calc_dependencies()`: Retrieves calculation dependencies
- `ReportWrapper().list_semantic_model_objects()`: Gets objects used in reports

These functions establish the foundation for analysis by gathering:
1. **Dependencies DataFrame**: Shows which objects reference other objects
2. **Report Objects DataFrame**: Lists objects actually used in report visualizations


In [ ]:
def find_model_dependencies():
    """
    Retrieves calculation dependencies from the semantic model.
    
    Uses fabric.get_model_calc_dependencies() to analyze which objects
    reference other objects within the model (e.g., measures referencing columns).
    
    Returns:
        pandas.DataFrame: Contains dependency relationships with columns:
            - Object: The object that has dependencies
            - Referenced Object: The object being referenced
            - Referenced Object Type: Type (Measure, Column, Table, etc.)
            - Referenced Table: Table containing the referenced object
    """
    dependencies = fabric.get_model_calc_dependencies(
        dataset=DATASET,
        workspace=WORKSPACE
    )

    with dependencies as calc_deps:
        df = getattr(calc_deps, "dependencies_df", None)

    return df


def find_report_objects():
    """
    Identifies objects used in report visualizations.
    
    Uses ReportWrapper to scan report pages and extract semantic model
    objects (measures, columns, tables) that are actively used in visuals.
    
    Returns:
        pandas.DataFrame: Contains report usage data with columns:
            - Object Type: Type of object (Measure, Column, Table)
            - Object Name: Name of the object
            - Table Name: Parent table name
            - Report Page: Page where object is used
            - Visual Type: Type of visual using the object
    """
    rpt = ReportWrapper(
        report=DATASET,
        workspace=WORKSPACE
    )

    report_objects = rpt.list_semantic_model_objects()

    return report_objects


# Execute data preparation functions
print("📊 Retrieving model dependencies...")
dependencies_df = find_model_dependencies()

print("📈 Analyzing report object usage...")
report_objects = find_report_objects()

print(f"✅ Found {len(dependencies_df)} dependency relationships")
print(f"✅ Found {len(report_objects)} objects used in reports")

# 🎯 Unused Measures Analysis

## How To Identify Unused Measures

### Process Overview:
1. **GET ALL MEASURES** - Extract complete measure inventory from model
2. **GET REFERENCED MEASURES** - Find measures referenced by other calculations
3. **GET MEASURES USED IN REPORT VISUALS** - Identify measures actively used in visualizations

### Libraries Used:
- `fabric.list_measures()`: Retrieves all measures from the semantic model
- Dependencies DataFrame: Filters for measures referenced in calculations
- Report Objects DataFrame: Filters for measures used in visualizations

### Analysis Logic:
```
UNUSED MEASURES = ALL MEASURES - (REFERENCED MEASURES ∪ REPORT MEASURES)
```

**A measure is considered unused if:**
- ❌ Not used in any report visualizations
- ❌ Not referenced by other calculated columns or measures
- ❌ Not used in calculated tables or other DAX expressions

---

In [ ]:
def find_unused_measures(deps_df, report_objects):
    """
    Identifies measures that are not used in reports or referenced by other objects.
    
    This function performs set operations to find measures that exist in the model
    but are neither used in report visualizations nor referenced by other
    calculated columns, measures, or DAX expressions.
    
    Args:
        deps_df (pandas.DataFrame): Dependencies dataframe from find_model_dependencies()
        report_objects (pandas.DataFrame): Report objects dataframe from find_report_objects()
    
    Returns:
        set: Set of unused measure names that can potentially be removed
    
    Process:
        1. Retrieves all measures using fabric.list_measures()
        2. Finds measures referenced in dependencies (Referenced Object Type = 'Measure')
        3. Finds measures used in report visuals (Object Type = 'Measure')
        4. Returns measures not in either referenced or report sets
    """
    
    # GET ALL MEASURES - Complete inventory from semantic model
    print("  📋 Retrieving all measures from semantic model...")
    measures_df = fabric.list_measures(
        dataset=DATASET, 
        workspace=WORKSPACE
    )

    all_measures = set(measures_df['Measure Name'].unique())
    print(f"     └─ Found {len(all_measures)} total measures")

    # GET REFERENCED MEASURES - Used in other calculations
    print("  🔗 Identifying measures referenced in calculations...")
    referenced_measures = set(
        deps_df[deps_df['Referenced Object Type'] == 'Measure']['Referenced Object'].unique()
    )
    print(f"     └─ Found {len(referenced_measures)} referenced measures")

    # GET MEASURES IN REPORT VISUALS - Actively used in reports
    print("  📊 Finding measures used in report visualizations...")
    report_measures = set(report_objects[report_objects['Object Type']=="Measure"]['Object Name'].unique())
    print(f"     └─ Found {len(report_measures)} measures in reports")

    # CALCULATE USED MEASURES - Union of referenced and report measures
    used_measures = report_measures.union(referenced_measures)
    print(f"  ✓ Total used measures: {len(used_measures)}")

    # RETURN UNUSED MEASURES - Set difference operation
    unused_measures = all_measures.difference(used_measures)
    print(f"  🎯 Identified {len(unused_measures)} unused measures")
    
    # Return both unused measures and metrics for visualization
    return unused_measures, {
        'total_measures': len(all_measures),
        'unused_measures': len(unused_measures),
        'used_measures': len(used_measures),
        'utilization_rate': (len(used_measures) / len(all_measures)) * 100 if len(all_measures) > 0 else 0
    }

In [ ]:
# Execute unused measures analysis
print("🔍 EXECUTING UNUSED MEASURES ANALYSIS...")
print("=" * 50)
unused_measures, measures_metrics = find_unused_measures(deps_df=dependencies_df, report_objects=report_objects)
print("=" * 50)

# Display results with enhanced formatting
if unused_measures:
    print(f"🚨 RESULTS: {len(unused_measures)} UNUSED MEASURES FOUND")
    print("\nThese measures can potentially be removed to optimize the model:")
    for i, measure in enumerate(sorted(unused_measures), 1):
        print(f"  {i:2d}. {measure}")
        
    print(f"\n💾 Potential storage savings: ~{len(unused_measures)} objects")
    print("⚡ Performance impact: Faster model processing and reduced memory usage")
else:
    print("✅ EXCELLENT! No unused measures found. Model is optimized.")

print("\n📊 Summary Statistics:")
print(f"   Total Measures: {measures_metrics['total_measures']}")
print(f"   Unused Measures: {measures_metrics['unused_measures']}")
print(f"   Used Measures: {measures_metrics['used_measures']}")
print(f"   Utilization Rate: {measures_metrics['utilization_rate']:.1f}%")

In [ ]:
# 📈 ENHANCED VISUALIZATION: Measures & Columns Analysis
print("\n📈 GENERATING DUAL UTILIZATION CHARTS...")

# Get columns data for dual visualization
total_columns = len(fabric.list_columns(dataset=DATASET, workspace=WORKSPACE))
unused_columns_for_viz = get_unused_columns(dependencies_df, report_objects)
used_columns = total_columns - len(unused_columns_for_viz)
columns_utilization = (used_columns / total_columns * 100) if total_columns > 0 else 0

# Create side-by-side charts
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# MEASURES CHART (Left)
measures_labels = ['Used', 'Unused']
measures_values = [measures_metrics['used_measures'], measures_metrics['unused_measures']]
measures_colors = ['#28a745', '#dc3545']  # Green for used, red for unused

bars1 = ax1.bar(measures_labels, measures_values, color=measures_colors, alpha=0.8, 
                 edgecolor='black', linewidth=1)

# Add value labels on measures bars
for i, bar in enumerate(bars1):
    height = bar.get_height()
    percentage = (measures_values[i] / measures_metrics['total_measures']) * 100
    ax1.annotate(f'{int(height)}\n({percentage:.1f}%)',
                 xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 5),
                 textcoords="offset points",
                 ha='center', va='bottom', 
                 fontsize=12, fontweight='bold')

# Customize measures chart
ax1.set_ylabel('Number of Measures', fontsize=13, fontweight='bold')
ax1.set_title(f'🎯 Measures Analysis\nTotal: {measures_metrics["total_measures"]} | Utilization: {measures_metrics["utilization_rate"]:.1f}%',
              fontsize=14, fontweight='bold', pad=15)
ax1.set_ylim(0, max(measures_values) * 1.3 if measures_values else 1)
ax1.grid(axis='y', alpha=0.3, linestyle='--')
ax1.set_axisbelow(True)

# COLUMNS CHART (Right)
columns_labels = ['Used', 'Unused']
columns_values = [used_columns, len(unused_columns_for_viz)]
columns_colors = ['#17a2b8', '#fd7e14']  # Blue for used, orange for unused

bars2 = ax2.bar(columns_labels, columns_values, color=columns_colors, alpha=0.8, 
                 edgecolor='black', linewidth=1)

# Add value labels on columns bars
for i, bar in enumerate(bars2):
    height = bar.get_height()
    percentage = (columns_values[i] / total_columns) * 100 if total_columns > 0 else 0
    ax2.annotate(f'{int(height)}\n({percentage:.1f}%)',
                 xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 5),
                 textcoords="offset points",
                 ha='center', va='bottom', 
                 fontsize=12, fontweight='bold')

# Customize columns chart
ax2.set_ylabel('Number of Columns', fontsize=13, fontweight='bold')
ax2.set_title(f'📊 Columns Analysis\nTotal: {total_columns} | Utilization: {columns_utilization:.1f}%',
              fontsize=14, fontweight='bold', pad=15)
ax2.set_ylim(0, max(columns_values) * 1.3 if columns_values else 1)
ax2.grid(axis='y', alpha=0.3, linestyle='--')
ax2.set_axisbelow(True)

# Overall chart formatting
fig.suptitle(f'🚀 {DATASET} - Model Optimization Analysis', 
             fontsize=16, fontweight='bold', y=0.95)

# Style both charts
for ax in [ax1, ax2]:
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=11)

plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Make room for main title
plt.show()

print("✅ Dual charts generated successfully!")
print(f"📊 Summary: {measures_metrics['used_measures']}/{measures_metrics['total_measures']} measures used, {used_columns}/{total_columns} columns used")

# 📊 Unused Columns Analysis

## How To Identify Unused Columns

### Process Overview:
1. **GET ALL COLUMNS** - Extract complete column inventory from all tables
2. **GET REFERENCED COLUMNS** - Find columns referenced in calculations and measures
3. **GET COLUMNS USED IN REPORTS** - Identify columns actively used in visualizations

### Libraries Used:
- `fabric.list_columns()`: Retrieves all columns from the semantic model
- Dependencies DataFrame: Filters for Column and Calc Column object types
- Report Objects DataFrame: Filters for columns used in visualizations

### Analysis Logic:
```
UNUSED COLUMNS = ALL COLUMNS - (REFERENCED COLUMNS ∪ REPORT COLUMNS)
```

**A column is considered unused if:**
- ❌ Not used in any report visualizations (filters, axes, legends, values)
- ❌ Not referenced by calculated columns or measures
- ❌ Not used in relationships (foreign/primary keys)
- ❌ Not used in row-level security expressions

**⚠️ Important Considerations:**
- Some columns may be used in external reports or applications
- Key columns for relationships should be retained even if not directly used
- Consider business requirements before removing columns

---

In [ ]:
def get_unused_columns(deps_df, report_objects):
    """
    Identifies columns that are not used in reports or referenced by other objects.
    
    This function analyzes both regular columns and calculated columns to determine
    which ones are not being utilized in the semantic model or reports.
    
    Args:
        deps_df (pandas.DataFrame): Dependencies dataframe from find_model_dependencies()
        report_objects (pandas.DataFrame): Report objects dataframe from find_report_objects()
    
    Returns:
        set: Set of unused column names that can potentially be removed
    
    Process:
        1. Retrieves all columns using fabric.list_columns()
        2. Finds columns referenced in dependencies (both Column and Calc Column types)
        3. Finds columns used in report visuals
        4. Returns columns not in either referenced or report sets
    """
    
    # GET ALL COLUMNS - Complete inventory from all tables
    print("  📋 Retrieving all columns from semantic model...")
    columns_df = fabric.list_columns(
        dataset=DATASET, 
        workspace=WORKSPACE
        )
    all_columns = set(columns_df['Column Name'].unique())
    print(f"     └─ Found {len(all_columns)} total columns across all tables")

    # GET REFERENCED COLUMNS - Used in calculations
    print("  🔗 Identifying columns referenced in calculations...")
    col_object_types = ["Column", "Calc Column"]  # Include both regular and calculated columns
    referenced_columns = set(
        deps_df[deps_df['Referenced Object Type'].isin(col_object_types)]['Referenced Object'].unique()
    )
    print(f"     └─ Found {len(referenced_columns)} referenced columns")

    # GET COLUMNS USED DIRECTLY IN REPORTS - Active in visualizations
    print("  📊 Finding columns used in report visualizations...")
    report_columns = set(report_objects[report_objects['Object Type'] == "Column"]['Object Name'].unique())
    print(f"     └─ Found {len(report_columns)} columns in reports")

    # CALCULATE USED COLUMNS - Union of referenced and report columns
    used_columns = report_columns.union(referenced_columns)
    print(f"  ✓ Total used columns: {len(used_columns)}")

    # RETURN UNUSED COLUMNS - Set difference operation
    unused_columns = all_columns.difference(used_columns)
    print(f"  🎯 Identified {len(unused_columns)} unused columns")
    
    return unused_columns

In [ ]:
# Execute unused columns analysis
print("🔍 EXECUTING UNUSED COLUMNS ANALYSIS...")
print("=" * 50)
unused_columns = get_unused_columns(deps_df=dependencies_df, report_objects=report_objects)
print("=" * 50)

# Display results with enhanced formatting
if unused_columns:
    print(f"🚨 RESULTS: {len(unused_columns)} UNUSED COLUMNS FOUND")
    print("\nThese columns can potentially be removed to optimize the model:")
    
    # Group columns by likely category for better organization
    date_cols = [col for col in unused_columns if any(x in col.lower() for x in ['date', 'week', 'month', 'quarter', 'year', 'day'])]
    financial_cols = [col for col in unused_columns if any(x in col.lower() for x in ['cost', 'sales', 'profit', 'amount', 'balance', 'vat'])]
    other_cols = [col for col in unused_columns if col not in date_cols + financial_cols]
    
    if date_cols:
        print(f"\n  📅 Date/Time Columns ({len(date_cols)}):")
        for col in sorted(date_cols)[:10]:  # Show first 10
            print(f"     • {col}")
        if len(date_cols) > 10: print(f"     ... and {len(date_cols)-10} more")
    
    if financial_cols:
        print(f"\n  💰 Financial Columns ({len(financial_cols)}):")
        for col in sorted(financial_cols):
            print(f"     • {col}")
    
    if other_cols:
        print(f"\n  🔧 Other Columns ({len(other_cols)}):")
        for col in sorted(other_cols)[:15]:  # Show first 15
            print(f"     • {col}")
        if len(other_cols) > 15: print(f"     ... and {len(other_cols)-15} more")
        
    print(f"\n💾 Potential storage savings: ~{len(unused_columns)} columns")
    print("⚡ Performance impact: Faster queries and reduced memory usage")
    print("⚠️  Note: Review relationship keys before removing columns")
else:
    print("✅ EXCELLENT! No unused columns found. Model is optimized.")

print("\n📊 Summary Statistics:")
total_columns = len(fabric.list_columns(dataset=DATASET, workspace=WORKSPACE))
print(f"   Total Columns: {total_columns}")
print(f"   Unused Columns: {len(unused_columns)}")
print(f"   Utilization Rate: {((total_columns - len(unused_columns)) / total_columns * 100):.1f}%")

# 🗃️ Unused Tables Analysis

## How To Identify Unused Tables

### Process Overview:
1. **GET ALL TABLES** - Extract complete table inventory from the semantic model
2. **GET REFERENCED TABLES** - Find tables referenced in calculations and measures
3. **GET TABLES USED IN REPORTS** - Identify tables with objects used in visualizations

### Libraries Used:
- `fabric.list_tables()`: Retrieves all tables from the semantic model
- Dependencies DataFrame: Extracts Referenced Table column values
- Report Objects DataFrame: Extracts Table Name column values

### Analysis Logic:
```
UNUSED TABLES = ALL TABLES - (REFERENCED TABLES ∪ REPORT TABLES)
```

**A table is considered unused if:**
- ❌ No columns from the table are used in report visualizations
- ❌ No measures reference columns from this table
- ❌ No calculated columns reference this table
- ❌ No relationships connect this table to active parts of the model

**⚠️ Critical Considerations:**
- **Review relationships carefully** - Key tables may be needed for joins
- **Check for external dependencies** - May be used by other reports/apps
- **Verify business requirements** - Some tables may be needed for future use
- **Consider bridge tables** - May be essential for many-to-many relationships

**💡 Safe Removal Criteria:**
- Table has no active relationships
- No external reports depend on this table
- Business stakeholders confirm it's no longer needed

---

In [ ]:
def get_unused_tables(deps_df, report_objects):
    """
    Identifies tables that are not used in reports or referenced by other objects.
    
    This function analyzes table usage across the entire semantic model to identify
    tables that have no active references or report usage.
    
    Args:
        deps_df (pandas.DataFrame): Dependencies dataframe from find_model_dependencies()
        report_objects (pandas.DataFrame): Report objects dataframe from find_report_objects()
    
    Returns:
        set: Set of unused table names that can potentially be removed
    
    Process:
        1. Retrieves all tables using fabric.list_tables()
        2. Finds tables referenced in dependencies (Referenced Table column)
        3. Finds tables with objects used in report visuals (Table Name column)
        4. Returns tables not in either referenced or report sets
    
    Warning:
        Exercise extreme caution when removing tables. Verify relationships
        and external dependencies before deletion.
    """
    
    # GET ALL TABLES - Complete inventory from semantic model
    print("  📋 Retrieving all tables from semantic model...")
    tables_df = fabric.list_tables(
        dataset=DATASET, 
        workspace=WORKSPACE
    )
    all_tables = set(tables_df['Name'].unique())
    print(f"     └─ Found {len(all_tables)} total tables")

    # GET REFERENCED TABLES - Used in calculations
    print("  🔗 Identifying tables referenced in calculations...")
    referenced_tables = set(
        deps_df['Referenced Table'].dropna().unique()
    )
    print(f"     └─ Found {len(referenced_tables)} referenced tables")

    # GET TABLES USED DIRECTLY IN REPORTS - Have objects in visualizations
    print("  📊 Finding tables with objects used in reports...")
    report_tables = set(
        report_objects['Table Name'].unique()
    )
    print(f"     └─ Found {len(report_tables)} tables used in reports")

    # CALCULATE USED TABLES - Union of referenced and report tables
    used_tables = referenced_tables.union(report_tables)
    print(f"  ✓ Total used tables: {len(used_tables)}")

    # RETURN UNUSED TABLES - Set difference operation
    unused_tables = all_tables.difference(used_tables)
    print(f"  🎯 Identified {len(unused_tables)} unused tables")
    
    return unused_tables

In [ ]:
# Execute unused tables analysis
print("🔍 EXECUTING UNUSED TABLES ANALYSIS...")
print("=" * 50)
unused_tables = get_unused_tables(deps_df=dependencies_df, report_objects=report_objects)
print("=" * 50)

# Display results with enhanced formatting and warnings
if unused_tables:
    print(f"🚨 RESULTS: {len(unused_tables)} UNUSED TABLES FOUND")
    print("\n⚠️  CRITICAL WARNING: Review these tables carefully before removal!")
    print("\nPotentially unused tables:")
    for i, table in enumerate(sorted(unused_tables), 1):
        print(f"  {i:2d}. {table}")
        
    print("\n🔍 RECOMMENDED VERIFICATION STEPS:")
    print("   1. Check if tables are used in relationships (even inactive ones)")
    print("   2. Verify no external reports or applications use these tables")
    print("   3. Confirm with business stakeholders before removal")
    print("   4. Consider if tables are needed for future development")
    print("   5. Review table relationships and cardinality")
        
    print(f"\n💾 Potential storage savings: ~{len(unused_tables)} tables")
    print("⚡ Performance impact: Reduced model complexity and faster refresh")
else:
    print("✅ EXCELLENT! No unused tables found. Model structure is optimized.")

print("\n📊 Summary Statistics:")
total_tables = len(fabric.list_tables(dataset=DATASET, workspace=WORKSPACE))
print(f"   Total Tables: {total_tables}")
print(f"   Unused Tables: {len(unused_tables)}")
print(f"   Utilization Rate: {((total_tables - len(unused_tables)) / total_tables * 100):.1f}%")

# 🔗 Relationship Analysis

## Model Relationships Overview

Understanding table relationships is crucial for validating unused table analysis.
Tables that appear "unused" might still be essential for maintaining proper
relationships and data integrity.

### Libraries Used:
- `fabric.list_relationships()`: Retrieves all relationships with extended properties

### Key Relationship Properties:
- **Multiplicity**: Defines the relationship type (1:1, 1:*, *:*)
- **Active**: Whether the relationship is active for filtering
- **Cross Filtering Behavior**: Direction of filter propagation
- **Cardinality**: Number of unique values in relationship columns

---

In [ ]:
# Analyze model relationships to understand table dependencies
print("🔗 ANALYZING MODEL RELATIONSHIPS...")
print("=" * 50)

# GET ALL RELATIONSHIPS with extended properties
relationships_df = fabric.list_relationships(
    workspace = WORKSPACE,
    dataset = DATASET,
    extended=True  # Include cardinality and other extended properties
)

print(f"📊 Found {len(relationships_df)} relationships in the model")

# Analyze relationship patterns
active_relationships = relationships_df[relationships_df['Active'] == True]
inactive_relationships = relationships_df[relationships_df['Active'] == False]

print(f"✅ Active relationships: {len(active_relationships)}")
print(f"❌ Inactive relationships: {len(inactive_relationships)}")

# Get unique tables involved in relationships
tables_in_relationships = set(relationships_df['From Table'].unique()) | set(relationships_df['To Table'].unique())
print(f"🗃️  Tables involved in relationships: {len(tables_in_relationships)}")

# Cross-reference with unused tables
if unused_tables:
    unused_tables_with_relationships = unused_tables.intersection(tables_in_relationships)
    if unused_tables_with_relationships:
        print(f"\n⚠️  WARNING: {len(unused_tables_with_relationships)} unused tables have relationships!")
        print("   These tables should be reviewed carefully:")
        for table in sorted(unused_tables_with_relationships):
            print(f"     • {table}")
    else:
        print("\n✅ No unused tables have active relationships - safer to remove")

print("\n📋 Relationship Summary Table:")
display(relationships_df)

# 🚀 Complete Model Analysis Execution

## Main Analysis Function

The `run_complete_analysis()` function executes all analysis steps in sequence:
1. **Data Preparation** - Gathers dependencies and report objects
2. **Measures Analysis** - Identifies unused measures with visualization
3. **Columns Analysis** - Finds unused columns with categorization
4. **Tables Analysis** - Locates unused tables with warnings
5. **Relationships Analysis** - Reviews model relationships
6. **Dual Visualization** - Shows measures AND columns side-by-side
7. **Summary Report** - Provides optimization recommendations

**Benefits of using the main function:**
- All results displayed in a single output
- Consistent execution flow
- Easy to track progress
- Professional reporting format
- Side-by-side visualization of measures and columns

---

In [ ]:
def run_complete_analysis():
    """
    Execute complete Fabric semantic model analysis.
    
    This main function orchestrates all analysis steps:
    1. Data preparation (dependencies and report objects)
    2. Unused measures analysis with visualization
    3. Unused columns analysis with categorization
    4. Unused tables analysis with warnings
    5. Relationships analysis for validation
    6. Dual visualization (measures + columns charts)
    7. Summary report with recommendations
    
    Returns:
        dict: Complete analysis results with all metrics and findings
    """
    
    print('🚀' + '=' * 70)
    print("🎯 FABRIC SEMANTIC MODEL OPTIMIZATION ANALYSIS")
    print("=" * 72)
    print(f"📊 Workspace: {WORKSPACE}")
    print(f"📈 Dataset: {DATASET}")
    print("=" * 72)


    
    # STEP 1: DATA PREPARATION
    print("📋 STEP 1: DATA PREPARATION")
    print("-" * 40)
    print("📊 Retrieving model dependencies...")
    dependencies_df = find_model_dependencies()
    
    print("📈 Analyzing report object usage...")
    report_objects = find_report_objects()
    
    print(f"✅ Found {len(dependencies_df)} dependency relationships")
    print(f"✅ Found {len(report_objects)} objects used in reports\n")
    
    # STEP 2: MEASURES ANALYSIS
    print("🎯 STEP 2: UNUSED MEASURES ANALYSIS")
    print("-" * 40)
    unused_measures, measures_metrics = find_unused_measures(dependencies_df, report_objects)
    
    # Display measures results
    if unused_measures:
        print(f"🚨 RESULTS: {len(unused_measures)} UNUSED MEASURES FOUND")
        print("\nUnused measures that can be removed:")
        for i, measure in enumerate(sorted(unused_measures), 1):
            print(f"  {i:2d}. {measure}")
        print(f"\n💾 Storage savings: ~{len(unused_measures)} objects")
    else:
        print("✅ EXCELLENT! No unused measures found.")
    
    print(f"📊 Measures Summary: {measures_metrics['used_measures']}/{measures_metrics['total_measures']} used ({measures_metrics['utilization_rate']:.1f}% utilization)\n")
    
    # STEP 3: COLUMNS ANALYSIS
    print("📊 STEP 3: UNUSED COLUMNS ANALYSIS")
    print("-" * 40)
    unused_columns = get_unused_columns(dependencies_df, report_objects)
    
    # Categorize and display columns results
    if unused_columns:
        print(f"🚨 RESULTS: {len(unused_columns)} UNUSED COLUMNS FOUND")
        
        # Group columns by category
        date_cols = [col for col in unused_columns if any(x in col.lower() for x in ['date', 'week', 'month', 'quarter', 'year', 'day'])]
        financial_cols = [col for col in unused_columns if any(x in col.lower() for x in ['cost', 'sales', 'profit', 'amount', 'balance', 'vat'])]
        other_cols = [col for col in unused_columns if col not in date_cols + financial_cols]
        
        if date_cols:
            print(f"\n📅 Date/Time Columns ({len(date_cols)}):")
            for col in sorted(date_cols)[:5]: print(f"   • {col}")
            if len(date_cols) > 5: print(f"   ... and {len(date_cols)-5} more")
        
        if financial_cols:
            print(f"\n💰 Financial Columns ({len(financial_cols)}):")
            for col in sorted(financial_cols): print(f"   • {col}")
        
        if other_cols:
            print(f"\n🔧 Other Columns ({len(other_cols)}):")
            for col in sorted(other_cols)[:5]: print(f"   • {col}")
            if len(other_cols) > 5: print(f"   ... and {len(other_cols)-5} more")
        
        print(f"💾 Storage savings: ~{len(unused_columns)} columns")
        print("⚠️  Note: Review relationship keys before removal")
    else:
        print("✅ EXCELLENT! No unused columns found.")
    
    total_columns = len(fabric.list_columns(dataset=DATASET, workspace=WORKSPACE))
    columns_utilization = ((total_columns - len(unused_columns)) / total_columns * 100) if total_columns > 0 else 0
    print(f"📊 Columns Summary: {total_columns - len(unused_columns)}/{total_columns} used ({columns_utilization:.1f}% utilization)\n")
    
    # STEP 4: TABLES ANALYSIS
    print("🗃️ STEP 4: UNUSED TABLES ANALYSIS")
    print("-" * 40)
    unused_tables = get_unused_tables(dependencies_df, report_objects)
    
    # Display tables results with warnings
    if unused_tables:
        print(f"🚨 RESULTS: {len(unused_tables)} UNUSED TABLES FOUND")
        print("⚠️  CRITICAL WARNING: Review carefully before removal!")
        print("\nPotentially unused tables:")
        for i, table in enumerate(sorted(unused_tables), 1):
            print(f"  {i:2d}. {table}")
        print(f"\n💾 Storage savings: ~{len(unused_tables)} tables")
    else:
        print("✅ EXCELLENT! No unused tables found.")
    
    total_tables = len(fabric.list_tables(dataset=DATASET, workspace=WORKSPACE))
    tables_utilization = ((total_tables - len(unused_tables)) / total_tables * 100) if total_tables > 0 else 0
    print(f"📊 Tables Summary: {total_tables - len(unused_tables)}/{total_tables} used ({tables_utilization:.1f}% utilization)\n")
    
    # STEP 5: RELATIONSHIPS ANALYSIS
    print("🔗 STEP 5: RELATIONSHIPS ANALYSIS")
    print("-" * 40)
    relationships_df = fabric.list_relationships(workspace=WORKSPACE, dataset=DATASET, extended=True)
    active_relationships = relationships_df[relationships_df['Active'] == True]
    inactive_relationships = relationships_df[relationships_df['Active'] == False]
    
    print(f"📊 Found {len(relationships_df)} total relationships")
    print(f"✅ Active relationships: {len(active_relationships)}")
    print(f"❌ Inactive relationships: {len(inactive_relationships)}")
    
    # Check unused tables with relationships
    if unused_tables:
        tables_in_relationships = set(relationships_df['From Table'].unique()) | set(relationships_df['To Table'].unique())
        unused_tables_with_relationships = unused_tables.intersection(tables_in_relationships)
        if unused_tables_with_relationships:
            print(f"⚠️  WARNING: {len(unused_tables_with_relationships)} unused tables have relationships!")
            for table in sorted(unused_tables_with_relationships):
                print(f"     • {table}")
        else:
            print("✅ No unused tables have relationships - safer to remove")
    print()
    
    # STEP 6: ENHANCED DUAL VISUALIZATION
    print("📈 STEP 6: GENERATING DUAL VISUALIZATIONS")
    print("-" * 40)
    
    # Create side-by-side visualizations for measures and columns
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
    
    # MEASURES CHART (Left)
    measures_labels = ['Used', 'Unused']
    measures_values = [measures_metrics['used_measures'], measures_metrics['unused_measures']]
    measures_colors = ['#28a745', '#dc3545']  # Green for used, red for unused
    
    bars1 = ax1.bar(measures_labels, measures_values, color=measures_colors, alpha=0.8, 
                     edgecolor='black', linewidth=1)
    
    # Add value labels on measures bars
    for i, bar in enumerate(bars1):
        height = bar.get_height()
        percentage = (measures_values[i] / measures_metrics['total_measures']) * 100
        ax1.annotate(f'{int(height)}\n({percentage:.1f}%)',
                     xy=(bar.get_x() + bar.get_width()/2, height),
                     xytext=(0, 5),
                     textcoords="offset points",
                     ha='center', va='bottom', 
                     fontsize=12, fontweight='bold')
    
    # Customize measures chart
    ax1.set_ylabel('Number of Measures', fontsize=13, fontweight='bold')
    ax1.set_title(f'🎯 Measures Analysis\nTotal: {measures_metrics["total_measures"]} | Utilization: {measures_metrics["utilization_rate"]:.1f}%',
                  fontsize=14, fontweight='bold', pad=15)
    ax1.set_ylim(0, max(measures_values) * 1.3)
    ax1.grid(axis='y', alpha=0.3, linestyle='--')
    ax1.set_axisbelow(True)
    
    # COLUMNS CHART (Right)
    used_columns = total_columns - len(unused_columns)
    columns_labels = ['Used', 'Unused']
    columns_values = [used_columns, len(unused_columns)]
    columns_colors = ['#17a2b8', '#fd7e14']  # Blue for used, orange for unused
    
    bars2 = ax2.bar(columns_labels, columns_values, color=columns_colors, alpha=0.8, 
                     edgecolor='black', linewidth=1)
    
    # Add value labels on columns bars
    for i, bar in enumerate(bars2):
        height = bar.get_height()
        percentage = (columns_values[i] / total_columns) * 100 if total_columns > 0 else 0
        ax2.annotate(f'{int(height)}\n({percentage:.1f}%)',
                     xy=(bar.get_x() + bar.get_width()/2, height),
                     xytext=(0, 5),
                     textcoords="offset points",
                     ha='center', va='bottom', 
                     fontsize=12, fontweight='bold')
    
    # Customize columns chart
    ax2.set_ylabel('Number of Columns', fontsize=13, fontweight='bold')
    ax2.set_title(f'📊 Columns Analysis\nTotal: {total_columns} | Utilization: {columns_utilization:.1f}%',
                  fontsize=14, fontweight='bold', pad=15)
    ax2.set_ylim(0, max(columns_values) * 1.3 if columns_values else 1)
    ax2.grid(axis='y', alpha=0.3, linestyle='--')
    ax2.set_axisbelow(True)
    
    # Overall chart formatting
    fig.suptitle(f'🚀 {DATASET} - Model Optimization Analysis', 
                 fontsize=16, fontweight='bold', y=0.95)
    
    # Style both charts
    for ax in [ax1, ax2]:
        ax.tick_params(axis='x', labelsize=12)
        ax.tick_params(axis='y', labelsize=11)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)  # Make room for main title
    plt.show()
    
    print("✅ Dual visualization generated successfully!")
    
    # FINAL SUMMARY
    print("\n" + "=" * 72)
    print("🎯 ANALYSIS COMPLETE - OPTIMIZATION SUMMARY")
    print("=" * 72)
    print(f"📊 Measures: {measures_metrics['unused_measures']}/{measures_metrics['total_measures']} unused ({100-measures_metrics['utilization_rate']:.1f}%)")
    print(f"📊 Columns:  {len(unused_columns)}/{total_columns} unused ({100-columns_utilization:.1f}%)")
    print(f"📊 Tables:   {len(unused_tables)}/{total_tables} unused ({100-tables_utilization:.1f}%)")
    print(f"📊 Relationships: {len(active_relationships)} active, {len(inactive_relationships)} inactive")
    
    total_unused_objects = len(unused_measures) + len(unused_columns) + len(unused_tables)
    if total_unused_objects > 0:
        print(f"\n💾 OPTIMIZATION POTENTIAL: {total_unused_objects} unused objects found")
        print("⚡ Expected benefits: Faster queries, reduced memory, simplified model")
        print("⚠️  Recommendation: Review and remove unused objects safely")
    else:
        print("\n✅ MODEL IS OPTIMIZED: No unused objects found!")
        print("🎉 Your semantic model is efficiently structured.")
    
    print("=" * 72)
    print("🚀 Analysis completed successfully!")
    print("=" * 72)
    
    # Return results for further processing if needed
    return {
        'unused_measures': unused_measures,
        'unused_columns': unused_columns,
        'unused_tables': unused_tables,
        'measures_metrics': measures_metrics,
        'columns_metrics': {'total': total_columns, 'unused': len(unused_columns), 'utilization': columns_utilization},
        'tables_metrics': {'total': total_tables, 'unused': len(unused_tables), 'utilization': tables_utilization},
        'relationships_df': relationships_df,
        'dependencies_df': dependencies_df,
        'report_objects': report_objects
    }

## 🎬 Execute Complete Analysis

Run the cell below to execute the complete semantic model analysis.
All results will be displayed in a single, organized output with dual visualization.

In [ ]:
# 🚀 EXECUTE COMPLETE ANALYSIS WITH DUAL VISUALIZATION
# Run this cell to perform comprehensive semantic model analysis
# Features dual charts showing both measures and columns utilization

analysis_results = run_complete_analysis()